In [1]:
# import dependencies
import pandas as pd
import os
import re

In [2]:
# import files 
ulta_ingredients = pd.read_csv(r'ulta_unique_ingredients.csv')
unique_chems = pd.read_csv(r'Unique_Chem.csv', usecols=['ChemicalName'])
ulta = pd.read_csv(r'ulta_makeup.csv', usecols= ['brand', 'description', 
                                                 'rating', 'ingredients', 
                                                 'number_of_reviews', 'price_USD', 'number_of_ingredients'])

In [3]:
# create dataframes from csv files
ulta_ing_df = pd.DataFrame(ulta_ingredients)
unique_chems_df = pd.DataFrame(unique_chems)
ulta_df = pd.DataFrame(ulta)

In [4]:
# create list of most toxic chemicals
toxic_chems = ['Propylparaben','Fd&c','D&C','P-Phenylenediamine','Formaldehyde','Palm Oil','Paraben','Alkyl Parahydroxy Benzoates',
'Cyclotetrasiloxane','Octamethylcyclotetrasiloxane','Cylcopentasiloxane','Polydimethylsiloxan','Sodium Lauryl Sulfate', 
'Sls', 'Sodium Laureth Sulfate', 'Sles', 'Sodium Lauryl Sulfoacetate', 'Sodium Lauroyl Isoethionate', 'Sodium Lauroyl Taurate', 
'Sodium Cocoyl Isoethionate', 'Sodium Lauroyl Methyl Isoethionate', 'Sodium Lauroyl Sarcosinate', 
'Disodium Laureth Sulfosuccinate','Benzophenone', 'Paba', 'Avobenzone', 'Homosalate', 'Ethoxycinnmate', 'Oxybenzone']

In [9]:
# update ingredients from list to string
ulta_df['ingredients'] = ulta_df['ingredients'].astype(str)
ulta_df['ingredients list'] = ulta_df['ingredients'].str.extract('\[(.*?)\]')

# udpate number of ingredients and price to int
ulta_df = ulta_df.astype({"number_of_ingredients": int, 'price_USD': int})
ulta_df

,brand,description,rating,ingredients,number_of_ingredients,number_of_reviews,price_USD,ingredients list,Propylparaben,Fd&c,...,Sodium Cocoyl Isoethionate,Sodium Lauroyl Methyl Isoethionate,Sodium Lauroyl Sarcosinate,Disodium Laureth Sulfosuccinate,Benzophenone,Paba,Avobenzone,Homosalate,Ethoxycinnmate,Oxybenzone
0,Philosophy,purity-made-simple-one-step-facial-cleanser,4.6,"['Water', 'Sodium Lauroamphoacetate', 'Sodium ...",29,3881,36,"'Water', 'Sodium Lauroamphoacetate', 'Sodium T...",0,0,...,0,0,0,0,0,0,0,0,0,0
1,Dermalogica,special-cleansing-gel,4.6,"['Water/Aqua/Eau', 'Sodium Laureth Sulfate', '...",16,2202,39,"'Water/Aqua/Eau', 'Sodium Laureth Sulfate', 'D...",0,0,...,0,0,0,0,0,0,0,0,0,0
2,Clinique,take-day-off-cleansing-balm,4.7,"['Ethylhexyl Palmitate', 'Carthamus Tinctorius...",9,1793,30,"'Ethylhexyl Palmitate', 'Carthamus Tinctorius ...",0,0,...,0,0,0,0,0,0,0,0,0,0
3,Clinique,liquid-facial-soap-mild,4.5,"['Water / Aqua / Eau', 'Sodium Laureth Sulfate...",23,601,18,"'Water / Aqua / Eau', 'Sodium Laureth Sulfate'...",0,0,...,0,0,0,0,0,0,0,0,0,0
4,Philosophy,microdelivery-exfoliating-facial-wash,4.0,"['Aqua/Water/Eau', 'Solum Diatomeae/Diatomaceo...",31,672,28,"'Aqua/Water/Eau', 'Solum Diatomeae/Diatomaceou...",0,0,...,0,0,0,0,0,0,0,0,0,0
5,It Cosmetics,confidence-in-a-cleanser,4.5,"['Aqua/Water/Eau', 'Disodium Cocoamphodiacetat...",69,467,28,"'Aqua/Water/Eau', 'Disodium Cocoamphodiacetate...",0,0,...,0,0,0,0,0,0,0,0,0,0
6,Dermalogica,clearing-skin-wash,4.5,"['Salicylic Acid (0.5%)', 'Water (Aqua)', 'Sod...",37,910,39,"'Salicylic Acid (0.5%)', 'Water (Aqua)', 'Sodi...",0,0,...,0,0,0,0,0,0,0,0,0,0
7,Tula,purifying-face-cleanser,4.3,"['Aqua/Water/Eau', 'Cocamidopropyl Betaine', '...",43,225,28,"'Aqua/Water/Eau', 'Cocamidopropyl Betaine', 'P...",0,0,...,0,0,0,0,0,0,0,0,0,0
8,Clinique,acne-solutions-cleansing-foam,4.6,['Active: 2.0% Salicylic Acid. Inactive: Water...,31,1006,22,'Active: 2.0% Salicylic Acid. Inactive: Water ...,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Dermalogica,ultracalming-cleanser,0.0,"['Water/Aqua/Eau', 'Cetearyl Alcohol', 'Bisabo...",29,182,39,"'Water/Aqua/Eau', 'Cetearyl Alcohol', 'Bisabol...",0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
# remove decimal from rating
ulta_df['rating (0.0)'] = (ulta_df['rating']*10)
ulta_df = ulta_df.astype({'rating (0.0)': int})
ulta_df = ulta_df.drop(columns=['test'])
# ulta_df

,brand,description,rating,ingredients,number_of_ingredients,number_of_reviews,price_USD,ingredients list,Propylparaben,Fd&c,...,Sodium Lauroyl Sarcosinate,Disodium Laureth Sulfosuccinate,Benzophenone,Paba,Avobenzone,Homosalate,Ethoxycinnmate,Oxybenzone,sum,rating (0.0)
0,Philosophy,purity-made-simple-one-step-facial-cleanser,4.6,"['Water', 'Sodium Lauroamphoacetate', 'Sodium ...",29,3881,36,"'Water', 'Sodium Lauroamphoacetate', 'Sodium T...",1,0,...,0,0,0,0,0,0,0,0,1,46
1,Dermalogica,special-cleansing-gel,4.6,"['Water/Aqua/Eau', 'Sodium Laureth Sulfate', '...",16,2202,39,"'Water/Aqua/Eau', 'Sodium Laureth Sulfate', 'D...",1,0,...,0,0,0,0,0,0,0,0,2,46
2,Clinique,take-day-off-cleansing-balm,4.7,"['Ethylhexyl Palmitate', 'Carthamus Tinctorius...",9,1793,30,"'Ethylhexyl Palmitate', 'Carthamus Tinctorius ...",0,0,...,0,0,0,0,0,0,0,0,0,47
3,Clinique,liquid-facial-soap-mild,4.5,"['Water / Aqua / Eau', 'Sodium Laureth Sulfate...",23,601,18,"'Water / Aqua / Eau', 'Sodium Laureth Sulfate'...",0,0,...,0,0,0,0,0,0,0,0,1,45
4,Philosophy,microdelivery-exfoliating-facial-wash,4.0,"['Aqua/Water/Eau', 'Solum Diatomeae/Diatomaceo...",31,672,28,"'Aqua/Water/Eau', 'Solum Diatomeae/Diatomaceou...",0,0,...,0,0,0,0,0,0,0,0,0,40
5,It Cosmetics,confidence-in-a-cleanser,4.5,"['Aqua/Water/Eau', 'Disodium Cocoamphodiacetat...",69,467,28,"'Aqua/Water/Eau', 'Disodium Cocoamphodiacetate...",0,0,...,0,0,0,0,0,0,0,0,1,45
6,Dermalogica,clearing-skin-wash,4.5,"['Salicylic Acid (0.5%)', 'Water (Aqua)', 'Sod...",37,910,39,"'Salicylic Acid (0.5%)', 'Water (Aqua)', 'Sodi...",0,0,...,0,0,0,0,0,0,0,0,0,45
7,Tula,purifying-face-cleanser,4.3,"['Aqua/Water/Eau', 'Cocamidopropyl Betaine', '...",43,225,28,"'Aqua/Water/Eau', 'Cocamidopropyl Betaine', 'P...",0,0,...,0,0,0,0,0,0,0,0,0,43
8,Clinique,acne-solutions-cleansing-foam,4.6,['Active: 2.0% Salicylic Acid. Inactive: Water...,31,1006,22,'Active: 2.0% Salicylic Acid. Inactive: Water ...,0,0,...,0,0,0,0,0,0,0,0,0,46
9,Dermalogica,ultracalming-cleanser,0.0,"['Water/Aqua/Eau', 'Cetearyl Alcohol', 'Bisabo...",29,182,39,"'Water/Aqua/Eau', 'Cetearyl Alcohol', 'Bisabol...",0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
# create columns in the ulta df for each toxic chem
for chem in toxic_chems:
    ulta_df[chem] = 0

In [11]:
ulta_df

,brand,description,rating,ingredients,number_of_ingredients,number_of_reviews,price_USD,ingredients list,Propylparaben,Fd&c,...,Sodium Cocoyl Isoethionate,Sodium Lauroyl Methyl Isoethionate,Sodium Lauroyl Sarcosinate,Disodium Laureth Sulfosuccinate,Benzophenone,Paba,Avobenzone,Homosalate,Ethoxycinnmate,Oxybenzone
0,Philosophy,purity-made-simple-one-step-facial-cleanser,4.6,"['Water', 'Sodium Lauroamphoacetate', 'Sodium ...",29,3881,36,"'Water', 'Sodium Lauroamphoacetate', 'Sodium T...",0,0,...,0,0,0,0,0,0,0,0,0,0
1,Dermalogica,special-cleansing-gel,4.6,"['Water/Aqua/Eau', 'Sodium Laureth Sulfate', '...",16,2202,39,"'Water/Aqua/Eau', 'Sodium Laureth Sulfate', 'D...",0,0,...,0,0,0,0,0,0,0,0,0,0
2,Clinique,take-day-off-cleansing-balm,4.7,"['Ethylhexyl Palmitate', 'Carthamus Tinctorius...",9,1793,30,"'Ethylhexyl Palmitate', 'Carthamus Tinctorius ...",0,0,...,0,0,0,0,0,0,0,0,0,0
3,Clinique,liquid-facial-soap-mild,4.5,"['Water / Aqua / Eau', 'Sodium Laureth Sulfate...",23,601,18,"'Water / Aqua / Eau', 'Sodium Laureth Sulfate'...",0,0,...,0,0,0,0,0,0,0,0,0,0
4,Philosophy,microdelivery-exfoliating-facial-wash,4.0,"['Aqua/Water/Eau', 'Solum Diatomeae/Diatomaceo...",31,672,28,"'Aqua/Water/Eau', 'Solum Diatomeae/Diatomaceou...",0,0,...,0,0,0,0,0,0,0,0,0,0
5,It Cosmetics,confidence-in-a-cleanser,4.5,"['Aqua/Water/Eau', 'Disodium Cocoamphodiacetat...",69,467,28,"'Aqua/Water/Eau', 'Disodium Cocoamphodiacetate...",0,0,...,0,0,0,0,0,0,0,0,0,0
6,Dermalogica,clearing-skin-wash,4.5,"['Salicylic Acid (0.5%)', 'Water (Aqua)', 'Sod...",37,910,39,"'Salicylic Acid (0.5%)', 'Water (Aqua)', 'Sodi...",0,0,...,0,0,0,0,0,0,0,0,0,0
7,Tula,purifying-face-cleanser,4.3,"['Aqua/Water/Eau', 'Cocamidopropyl Betaine', '...",43,225,28,"'Aqua/Water/Eau', 'Cocamidopropyl Betaine', 'P...",0,0,...,0,0,0,0,0,0,0,0,0,0
8,Clinique,acne-solutions-cleansing-foam,4.6,['Active: 2.0% Salicylic Acid. Inactive: Water...,31,1006,22,'Active: 2.0% Salicylic Acid. Inactive: Water ...,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Dermalogica,ultracalming-cleanser,0.0,"['Water/Aqua/Eau', 'Cetearyl Alcohol', 'Bisabo...",29,182,39,"'Water/Aqua/Eau', 'Cetearyl Alcohol', 'Bisabol...",0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
# loop through each products ingredients list to see if the toxic chemicals are included 
for chem in toxic_chems: 
    for i in range(0, len(ulta_df)-1):
        if chem in ulta_df['ingredients list'][i]: 
            ulta_df[chem][i] = 1
        else:
            continue

C:\Users\Trisarah\Anaconda3\envs\dsi\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [13]:
ulta_df.dropna()

,brand,description,rating,ingredients,number_of_ingredients,number_of_reviews,price_USD,ingredients list,Propylparaben,Fd&c,...,Sodium Cocoyl Isoethionate,Sodium Lauroyl Methyl Isoethionate,Sodium Lauroyl Sarcosinate,Disodium Laureth Sulfosuccinate,Benzophenone,Paba,Avobenzone,Homosalate,Ethoxycinnmate,Oxybenzone
0,Philosophy,purity-made-simple-one-step-facial-cleanser,4.6,"['Water', 'Sodium Lauroamphoacetate', 'Sodium ...",29,3881,36,"'Water', 'Sodium Lauroamphoacetate', 'Sodium T...",1,0,...,0,0,0,0,0,0,0,0,0,0
1,Dermalogica,special-cleansing-gel,4.6,"['Water/Aqua/Eau', 'Sodium Laureth Sulfate', '...",16,2202,39,"'Water/Aqua/Eau', 'Sodium Laureth Sulfate', 'D...",1,0,...,0,0,0,0,0,0,0,0,0,0
2,Clinique,take-day-off-cleansing-balm,4.7,"['Ethylhexyl Palmitate', 'Carthamus Tinctorius...",9,1793,30,"'Ethylhexyl Palmitate', 'Carthamus Tinctorius ...",0,0,...,0,0,0,0,0,0,0,0,0,0
3,Clinique,liquid-facial-soap-mild,4.5,"['Water / Aqua / Eau', 'Sodium Laureth Sulfate...",23,601,18,"'Water / Aqua / Eau', 'Sodium Laureth Sulfate'...",0,0,...,0,0,0,0,0,0,0,0,0,0
4,Philosophy,microdelivery-exfoliating-facial-wash,4.0,"['Aqua/Water/Eau', 'Solum Diatomeae/Diatomaceo...",31,672,28,"'Aqua/Water/Eau', 'Solum Diatomeae/Diatomaceou...",0,0,...,0,0,0,0,0,0,0,0,0,0
5,It Cosmetics,confidence-in-a-cleanser,4.5,"['Aqua/Water/Eau', 'Disodium Cocoamphodiacetat...",69,467,28,"'Aqua/Water/Eau', 'Disodium Cocoamphodiacetate...",0,0,...,0,0,0,0,0,0,0,0,0,0
6,Dermalogica,clearing-skin-wash,4.5,"['Salicylic Acid (0.5%)', 'Water (Aqua)', 'Sod...",37,910,39,"'Salicylic Acid (0.5%)', 'Water (Aqua)', 'Sodi...",0,0,...,0,0,0,0,0,0,0,0,0,0
7,Tula,purifying-face-cleanser,4.3,"['Aqua/Water/Eau', 'Cocamidopropyl Betaine', '...",43,225,28,"'Aqua/Water/Eau', 'Cocamidopropyl Betaine', 'P...",0,0,...,0,0,0,0,0,0,0,0,0,0
8,Clinique,acne-solutions-cleansing-foam,4.6,['Active: 2.0% Salicylic Acid. Inactive: Water...,31,1006,22,'Active: 2.0% Salicylic Acid. Inactive: Water ...,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Dermalogica,ultracalming-cleanser,0.0,"['Water/Aqua/Eau', 'Cetearyl Alcohol', 'Bisabo...",29,182,39,"'Water/Aqua/Eau', 'Cetearyl Alcohol', 'Bisabol...",0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
# create list of the string coloumns from the ulta df
ulta_string_col = ['brand','description','rating','ingredients','number_of_reviews','price_USD','number_of_ingredients','ingredients list']

# drop the string columns
ulta_no_string_df = ulta_df.drop(columns=ulta_string_col)

In [15]:
# make the integer columns a list
ingredients_col_names = ulta_no_string_df.columns.tolist()

In [50]:
# save sum of rows(axis=1) in new column on ulta df
ulta_df['sum of toxic ingredients'] = ulta_df[ingredients_col_names].sum(axis=1)
# ulta_df = ulta_df.drop(columns='sum')
ulta_df

,brand,description,rating,ingredients,number_of_ingredients,number_of_reviews,price_USD,ingredients list,Propylparaben,Fd&c,...,Sodium Lauroyl Sarcosinate,Disodium Laureth Sulfosuccinate,Benzophenone,Paba,Avobenzone,Homosalate,Ethoxycinnmate,Oxybenzone,rating (0.0),sum of toxic ingredients
0,Philosophy,purity-made-simple-one-step-facial-cleanser,4.6,"['Water', 'Sodium Lauroamphoacetate', 'Sodium ...",29,3881,36,"'Water', 'Sodium Lauroamphoacetate', 'Sodium T...",1,0,...,0,0,0,0,0,0,0,0,46,1
1,Dermalogica,special-cleansing-gel,4.6,"['Water/Aqua/Eau', 'Sodium Laureth Sulfate', '...",16,2202,39,"'Water/Aqua/Eau', 'Sodium Laureth Sulfate', 'D...",1,0,...,0,0,0,0,0,0,0,0,46,2
2,Clinique,take-day-off-cleansing-balm,4.7,"['Ethylhexyl Palmitate', 'Carthamus Tinctorius...",9,1793,30,"'Ethylhexyl Palmitate', 'Carthamus Tinctorius ...",0,0,...,0,0,0,0,0,0,0,0,47,0
3,Clinique,liquid-facial-soap-mild,4.5,"['Water / Aqua / Eau', 'Sodium Laureth Sulfate...",23,601,18,"'Water / Aqua / Eau', 'Sodium Laureth Sulfate'...",0,0,...,0,0,0,0,0,0,0,0,45,1
4,Philosophy,microdelivery-exfoliating-facial-wash,4.0,"['Aqua/Water/Eau', 'Solum Diatomeae/Diatomaceo...",31,672,28,"'Aqua/Water/Eau', 'Solum Diatomeae/Diatomaceou...",0,0,...,0,0,0,0,0,0,0,0,40,0
5,It Cosmetics,confidence-in-a-cleanser,4.5,"['Aqua/Water/Eau', 'Disodium Cocoamphodiacetat...",69,467,28,"'Aqua/Water/Eau', 'Disodium Cocoamphodiacetate...",0,0,...,0,0,0,0,0,0,0,0,45,1
6,Dermalogica,clearing-skin-wash,4.5,"['Salicylic Acid (0.5%)', 'Water (Aqua)', 'Sod...",37,910,39,"'Salicylic Acid (0.5%)', 'Water (Aqua)', 'Sodi...",0,0,...,0,0,0,0,0,0,0,0,45,0
7,Tula,purifying-face-cleanser,4.3,"['Aqua/Water/Eau', 'Cocamidopropyl Betaine', '...",43,225,28,"'Aqua/Water/Eau', 'Cocamidopropyl Betaine', 'P...",0,0,...,0,0,0,0,0,0,0,0,43,0
8,Clinique,acne-solutions-cleansing-foam,4.6,['Active: 2.0% Salicylic Acid. Inactive: Water...,31,1006,22,'Active: 2.0% Salicylic Acid. Inactive: Water ...,0,0,...,0,0,0,0,0,0,0,0,46,0
9,Dermalogica,ultracalming-cleanser,0.0,"['Water/Aqua/Eau', 'Cetearyl Alcohol', 'Bisabo...",29,182,39,"'Water/Aqua/Eau', 'Cetearyl Alcohol', 'Bisabol...",0,0,...,0,0,0,0,0,0,0,0,0,0


In [51]:
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)
ulta_df.head()

,brand,description,rating,ingredients,number_of_ingredients,number_of_reviews,price_USD,ingredients list,Propylparaben,Fd&c,...,Sodium Lauroyl Sarcosinate,Disodium Laureth Sulfosuccinate,Benzophenone,Paba,Avobenzone,Homosalate,Ethoxycinnmate,Oxybenzone,rating (0.0),sum of toxic ingredients
0,Philosophy,purity-made-simple-one-step-facial-cleanser,4.6,"['Water', 'Sodium Lauroamphoacetate', 'Sodium ...",29,3881,36,"'Water', 'Sodium Lauroamphoacetate', 'Sodium T...",1,0,...,0,0,0,0,0,0,0,0,46,1
1,Dermalogica,special-cleansing-gel,4.6,"['Water/Aqua/Eau', 'Sodium Laureth Sulfate', '...",16,2202,39,"'Water/Aqua/Eau', 'Sodium Laureth Sulfate', 'D...",1,0,...,0,0,0,0,0,0,0,0,46,2
2,Clinique,take-day-off-cleansing-balm,4.7,"['Ethylhexyl Palmitate', 'Carthamus Tinctorius...",9,1793,30,"'Ethylhexyl Palmitate', 'Carthamus Tinctorius ...",0,0,...,0,0,0,0,0,0,0,0,47,0
3,Clinique,liquid-facial-soap-mild,4.5,"['Water / Aqua / Eau', 'Sodium Laureth Sulfate...",23,601,18,"'Water / Aqua / Eau', 'Sodium Laureth Sulfate'...",0,0,...,0,0,0,0,0,0,0,0,45,1
4,Philosophy,microdelivery-exfoliating-facial-wash,4.0,"['Aqua/Water/Eau', 'Solum Diatomeae/Diatomaceo...",31,672,28,"'Aqua/Water/Eau', 'Solum Diatomeae/Diatomaceou...",0,0,...,0,0,0,0,0,0,0,0,40,0


In [53]:
# save df as csv
ulta_df.to_csv('ulta_toxic_chemicals_data.csv', index=False)

In [ ]:
# ulta_df.sort_values(by='sum', ascending=False)